In [5]:
import json
import httpx

SENATE_URL = "https://legis.senado.leg.br/dadosabertos/"

In [6]:
with open("processos.json") as f:
    senatebills = json.load(f)

In [7]:
rels = []

async def get_relatoria(id_proc, rels, sem):
    if len(rels) > 10:
        return
    async with sem:
        async with httpx.AsyncClient() as client:
            print(id_proc)
            response = await client.get(f"{SENATE_URL}processo/relatoria", params={"idProcesso": id_proc})
            response.raise_for_status()
            data = response.json()
            if data:
                rels.append(data)
import asyncio
sem = asyncio.Semaphore(10)

tasks = [get_relatoria(bill['id'], rels, sem) for bill in senatebills]

result = await asyncio.gather(*tasks, return_exceptions=True)

1326805
2904181
2904162
3055426
3055179
3055502
3055445
3055217
3055407
3055350
3055160
3055464
3055483
3055027
3055065
3055084
3055103
3055122
3055046
3054666
3054780
3054685
3054628
3054932
3054799
3054856
3054875
3054913
3054704
3054647
3054818
3054761
3054742
3054894
3054552
3054951
2526648
2526572
3054514
3054533
2526553
3054455
3054495
2526515
2526439
2526420
3054436
2526363
2526344
3366005
3342354
1152635
3024590
3353512
3362211
3337791
3348668
3349440
3023185
3023223
3367457
3358769
3345932
3345740
3362599
3021541
3335825
3338720
3338176
3334216
3341488
3365041
3350512
3352770
3353576
3347994
3021233
1039738
3344289
3344579
3342834
3347898
3349472
3336661
3334280
3340394
3337013
3371246
3347832
3348538
3356767
3350836
3370445
3370671
3352224
3371408
3335889
3352996
3370703
3372952
3335633
3340716
3360156
3341520
3347192
3371150
3340266
3337727
3340812
3340780
3332480
3337141
3339396
3332576
3332832
3335665
3351934
3335279
3365301
3349408
3340104
3370639
3343092
3343609
3358315


In [8]:
rels[0]

[{'id': 7663544,
  'casaRelator': 'SF',
  'idTipoRelator': 1,
  'descricaoTipoRelator': 'Relator',
  'dataDesignacao': '2015-10-08 00:00:00',
  'dataDestituicao': '2016-03-30 00:00:00',
  'descricaoTipoEncerramento': 'Alteração na composição da comissão',
  'idProcesso': 2904181,
  'numeroAutuacao': '1',
  'codigoMateria': 236,
  'identificacaoProcesso': 'MSG 31/1991',
  'ementaProcesso': 'Encaminha ao Congresso Nacional, as Contas do Governo Federal, reletivas ao exercício financeiro de 1990.',
  'autoriaProcesso': 'Presidência da República',
  'dataApresentacaoProcesso': '1991-04-15 00:00:00',
  'tramitando': 'S',
  'codigoParlamentar': 5537,
  'nomeParlamentar': 'Dário Berger',
  'nomeCompleto': 'Dário Elias Berger',
  'sexoParlamentar': 'M',
  'formaTratamentoParlamentar': 'Senador',
  'urlFotoParlamentar': 'http://www.senado.leg.br/senadores/img/fotos-oficiais/senador5537.jpg',
  'urlPaginaParlamentar': 'http://www25.senado.leg.br/web/senadores/senador/-/perfil/5537',
  'emailParl